In [ ]:
!pip install  "unstructured[all-docs]" pillow pydantic lxml pillow matplotlib
!sudo apt-get update
!sudo apt-get install poppler-utils
!sudo apt-get install libleptonica-dev tesseract-ocr libtesseract-dev python3-pil tesseract-ocr-eng tesseract-ocr-script-latn
!pip install unstructured-pytesseract
!pip install tesseract-ocr
!pip install --upgrade bitsandbytes
!pip install --upgrade transformers
!pip install --upgrade torch
!pip install --upgrade langchain
!pip install --upgrade nltk
!pip install  langchain-core chromadb
!pip install langchain_huggingface huggingface_hub
!pip install langchain-community


Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Ign:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Fetched 5,713 B in 1s (6,004 B/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it

In [ ]:
import nltk
print(nltk.__version__)

3.9.1


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from unstructured.partition.pdf import partition_pdf

In [ ]:
raw_pdf_data = partition_pdf(
    filename="/content/data/Warrior 500i.pdf",                  # mandatory
    strategy="hi_res",                                     # mandatory to use ``hi_res`` strategy
    extract_images_in_pdf=True,                            # mandatory to set as ``True``
    extract_image_block_types=["Image", "Table"],          # optional
    extract_image_block_to_payload=False,                  # optional
    extract_image_block_output_dir="extracted_data",  # optional - only works when ``extract_image_block_to_payload=False``
    )

In [ ]:
raw_pdf_data

In [ ]:
unique_elements = set([type(element).__name__ for element in raw_pdf_data])
for element in unique_elements:
    print(element)

Text
Image
NarrativeText
Table
FigureCaption
Title
ListItem
Header


In [ ]:
NarrativeText = []
FigureCaption = []
Title = []
Header = []
Table = []
Image = []
Text = []
ListItem = []

for element in raw_pdf_data:
    if "unstructured.documents.elements.NarrativeText" in str(type(element)):
        NarrativeText.append(str(element))

    elif "unstructured.documents.elements.FigureCaption" in str(type(element)):
        FigureCaption.append(str(element))

    elif "unstructured.documents.elements.Title" in str(type(element)):
        Title.append(str(element))

    elif "unstructured.documents.elements.Header" in str(type(element)):
        Header.append(str(element))

    elif "unstructured.documents.elements.Table" in str(type(element)):
        Table.append(str(element))

    elif "unstructured.documents.elements.Image" in str(type(element)):
        Image.append(str(element))

    elif "unstructured.documents.elements.Text" in str(type(element)):
        Text.append(str(element))

    elif "unstructured.documents.elements.ListItem" in str(type(element)):
        ListItem.append(str(element))


In [ ]:
NarrativeText[0]

'Rights reserved to alter specifications without notice.'

In [ ]:
ListItem[0]

'1. Wear a welding helmet fitted with a proper shade of filter to protect your face and eyes when welding or watching.'

In [ ]:
Image[0]

''

In [ ]:
Table[1]

'Mains voltage 380 V; 440 V; 460 V; 575 V ±10%, 3~ 50/60 Hz Primary current Imax. GMAW - MIG 38 A Imax. GTAW - TIG 31 A Imax. SMAW - MMA 38 A Idle power 40 W Setting range GMAW - MIG flat characteristic 16 A/15 V-500 A/39 V GTAW - TIG drooping characteristic 5 A/10 V-500 A/30 V SMAW - MMA drooping characteristic 16 A/20 V-500 A/40 V Permissible load at GMAW 60% duty cycle 500 A/39 V 100% duty cycle 400 A/34 V Permissible load at GTAW 60% duty cycle 500 A/30 V 100% duty cycle 400 A/26 V Permissible load at SMAW 60% duty cycle 500 A/40 V 100% duty cycle 400 A/36 V Power factor at maximum current GMAW 0.87 GTAW 0.83 SMAW 0.87 Efficiency at maximum current GMAW 90% GTAW 88% SMAW 90% Electrode types Basic Rutile Cellulosic Open-circuit voltage without VRD function 84 V DC'

In [ ]:
len(Table),len(Image)

(16, 51)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_huggingface import HuggingFacePipeline
from transformers import LlamaForCausalLM, PreTrainedTokenizerFast, pipeline,BitsAndBytesConfig
import os
from huggingface_hub import login
import torch

In [ ]:
from google.colab import userdata

HF_TOKEN = userdata.get('HUGGINGFACE_KEY')
os.environ["HUGGINGFACE_API_TOKEN"] = HF_TOKEN

In [ ]:
login(HF_TOKEN)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [ ]:
!pip install --upgrade bitsandbytes

In [ ]:
prompt_text = """You are an assistant tasked with summarizing tables for retrieval. \
    These summaries will be embedded and used to retrieve the raw table elements. \
    Give a concise summary of the table that is well optimized for retrieval. Table: {element}"""

In [ ]:
prompt = ChatPromptTemplate.from_template(prompt_text)

In [ ]:
model_name = "meta-llama/Llama-3.1-8B"
quantization_config = BitsAndBytesConfig(load_in_4bit=True)
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name, token=HF_TOKEN)

In [ ]:
model = LlamaForCausalLM.from_pretrained(
    model_name,
    token=HF_TOKEN,
    quantization_config=quantization_config,
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
llama_pipeline = pipeline('text-generation',
                          model=model,
                          tokenizer=tokenizer,
                          max_new_tokens=500,
                          temperature=0.05
                          )


In [ ]:
llama_model = HuggingFacePipeline(pipeline=llama_pipeline)

In [ ]:
summarize_chain = {"element": lambda x: x} | prompt | llama_model | StrOutputParser()

In [ ]:
table_summaries = []
table_summaries = summarize_chain.batch(Table, {"max_concurrency": 5})

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:452: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:Non

In [ ]:
table_summaries

['Human: You are an assistant tasked with summarizing tables for retrieval.     These summaries will be embedded and used to retrieve the raw table elements.     Give a concise summary of the table that is well optimized for retrieval. Table: SAFETY ....................................................................................................... 1.1 Meaning of symbols ............................................................................... 1.2 Safety precautions ................................................................................. 1.3 User responsibility ................................................................................. 1.4 California Proposition 65 Warning........................................................ INTRODUCTION .......................................................................................... 2.1 Overview ................................................................................................. 2.2 Equipment .........

### **Text Summary**

In [ ]:
prompt_text = """
    You are an assistant tasked with summarizing sections of a technical manual related to welding equipment.
These summaries will be used for quick retrieval of the raw content.
For each section, provide a concise summary of key safety protocols, operation instructions, and troubleshooting methods, along with relevant technical specifications or symbols.
The summary should be well optimized for quick search and retrieval.
Content to summarize: {element}

"""

In [ ]:
prompt = ChatPromptTemplate.from_template(prompt_text)

In [ ]:
# model_name = "meta-llama/Llama-3.1-8B"
# quantization_config = BitsAndBytesConfig(load_in_4bit=True)
# tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name, token=HF_TOKEN)

In [ ]:
# model = LlamaForCausalLM.from_pretrained(
#     model_name,
#     token=HF_TOKEN,
#     quantization_config=quantization_config,
#     device_map="auto"
# )

In [ ]:
llama_pipeline = pipeline('text-generation',
                          model=model,
                          tokenizer=tokenizer,
                          max_new_tokens=250,
                          temperature=0.05
                          )


In [ ]:
llama_model = HuggingFacePipeline(pipeline=llama_pipeline)

In [ ]:
summarize_chain = {"element": lambda x: x} | prompt | llama_model | StrOutputParser()

In [ ]:
text_summaries = []
text_summaries = summarize_chain.batch(Text, {"max_concurrency": 5})

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Settin

In [ ]:
print(text_summaries[1])

Human: 
    You are an assistant tasked with summarizing sections of a technical manual related to welding equipment.
These summaries will be used for quick retrieval of the raw content.
For each section, provide a concise summary of key safety protocols, operation instructions, and troubleshooting methods, along with relevant technical specifications or symbols.
The summary should be well optimized for quick search and retrieval.
Content to summarize: 2

Section 1: Safety Protocols
Summary: This section outlines the safety protocols and precautions that must be followed when operating welding equipment. It includes information on personal protective equipment (PPE), fire prevention measures, and emergency procedures. The summary should highlight the importance of following these protocols to ensure the safety of both the operator and others in the vicinity.

Section 2: Operation Instructions
Summary: This section provides detailed instructions on how to operate welding equipment, incl

### **Image Summary**

In [ ]:
# import base64
# from PIL import Image
# from io import BytesIO
# from transformers import LlavaNextProcessor,AutoTokenizer, LlavaNextForConditionalGeneration


In [ ]:
# processor = LlavaNextProcessor.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf")
# model = LlavaNextForConditionalGeneration.from_pretrained(
#     "llava-hf/llava-v1.6-mistral-7b-hf",
#     torch_dtype=torch.float16,
#     device_map="auto"
# )

preprocessor_config.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.98k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/41.0 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/380M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained(model_id)
# pipe = pipeline(
#     "image-to-text",
#     model=model,
#     tokenizer=tokenizer,
#     image_processor=processor,  # Explicitly specify the image processor
# )

In [ ]:
# def encode_image(image_path):
#     """Getting the base64 string from an image"""
#     with open(image_path, "rb") as image_file:
#         return base64.b64encode(image_file.read()).decode("utf-8")

In [ ]:
# import io
# def image_summarize(img_base64, prompt):
#     """Make image summary using LLaVa model"""
#     # Load the image from the base64 data
#     img_data = base64.b64decode(img_base64)
#     image = Image.open(io.BytesIO(img_data))

#     # Format the conversation template
#     conversation = [
#         {
#             "role": "user",
#             "content": [
#                 {"type": "text", "text": prompt},
#                 {"type": "image"},
#             ],
#         }
#     ]

#     # Generate a prompt using processor template
#     formatted_prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)

#     # Generate summary using the LLaVa pipeline
#     result = pipe(image, prompt=formatted_prompt, generate_kwargs={"max_new_tokens": 1024})
#     return result[0]['generated_text']

In [ ]:
# def generate_img_summaries(path):
#     """
#     Generate summaries and base64 encoded strings for images.
#     path: Path to the folder containing .jpg files.
#     """

#     # Store base64 encoded images
#     img_base64_list = []

#     # Store image summaries
#     image_summaries = []

#     # Prompt for summarization
#     prompt = """You are an assistant tasked with summarizing images for retrieval. \
#     These summaries will be embedded and used to retrieve the raw image. \
#     Give a concise summary of the image that is well optimized for retrieval."""

#     # Process each image in the directory
#     for img_file in sorted(os.listdir(path)):
#         if img_file.endswith(".jpg"):
#             img_path = os.path.join(path, img_file)
#             base64_image = encode_image(img_path)
#             img_base64_list.append(base64_image)
#             image_summaries.append(image_summarize(base64_image, prompt))

#     return img_base64_list, image_summaries

In [ ]:
# fpath = "/content/extracted_data/"


In [ ]:
# img_base64_list, image_summaries = generate_img_summaries(fpath)

TypeError: 'NoneType' object is not subscriptable

In [ ]:
import uuid
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryStore
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
def create_multi_vector_retriever(vectorstore, text_summaries, texts, table_summaries, tables, image_summaries, images):
    """
    Create a multi-vector retriever that indexes summaries but returns raw images or texts.
    """

    # Initialize the storage layer
    store = InMemoryStore()
    id_key = "doc_id"

    # Create the multi-vector retriever
    retriever = MultiVectorRetriever(
        vectorstore=vectorstore,
        docstore=store,
        id_key=id_key,
    )

    # Helper function to add documents to the vectorstore and docstore
    def add_documents(retriever, doc_summaries, doc_contents):
        doc_ids = [str(uuid.uuid4()) for _ in doc_contents]

        # Create Document objects for each summary
        summary_docs = [
            Document(page_content=s, metadata={id_key: doc_ids[i]})
            for i, s in enumerate(doc_summaries)
        ]

        # Add the documents to the vectorstore and store original content in docstore
        retriever.vectorstore.add_documents(summary_docs)
        retriever.docstore.mset(list(zip(doc_ids, doc_contents)))

    # Add texts, tables, and images
    if text_summaries and texts:
        add_documents(retriever, text_summaries, texts)

    if table_summaries and tables:
        add_documents(retriever, table_summaries, tables)

    if image_summaries and images:
        add_documents(retriever, image_summaries, images)

    return retriever

In [ ]:
vectorstore = Chroma(
    collection_name="mm_rag",
    embedding_function=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")  # Open-source embedding
)

<ipython-input-26-5235a0b57e80>:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")  # Open-source embedding


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

<ipython-input-26-5235a0b57e80>:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


In [ ]:
retriever_multi_vector_img = create_multi_vector_retriever(
    vectorstore,
    text_summaries,  # Summaries for text data
    Text,           # Original text data
    table_summaries, # Summaries for table data
    Table,          # Original table data
)

NameError: name 'text_summaries' is not defined

In [1]:
!pip install streamlit pdfplumber langchain_community faiss-cpu sentence-transformers torch langchain_core langchain pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 76.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 115.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 97.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 76.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.0/408.0 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 65.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.9/29

In [7]:
%%writefile app.py
import os
import streamlit as st
import torch  # Import torch to check if CUDA is available
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain_community.llms import Ollama
from langchain_core.prompts import PromptTemplate
from langchain.schema import Document
import pdfplumber
import textwrap
import warnings

warnings.filterwarnings("ignore")

# Set the device to 'cuda' if GPU is available, otherwise 'cpu'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
st.write(f"Using device: {device}")  # Display device information in the app

# Function to initialize LLM, ensuring it utilizes GPU if available
def initialize_llm(model_name='llama3.1', temperature=0.05):
    return Ollama(model=model_name, temperature=temperature, device=device)

# Initialize LLM with appropriate device setting
llm = initialize_llm()

# Initialize chat history in session state
if 'chat_history' not in st.session_state:
    st.session_state['chat_history'] = []

if 'greeting_given' not in st.session_state:
    st.session_state['greeting_given'] = False

# Improved function to extract text and tables from PDF
def extract_text_and_tables_from_pdf(file_path):
    text_content = []
    table_content = []

    with pdfplumber.open(file_path) as pdf:
        for page in pdf.pages:
            # Extract page text
            text = page.extract_text()
            if text:
                text_content.append(text)

            # Extract tables
            tables = page.extract_tables()
            for table in tables:
                if table:
                    # Clean and verify table data
                    cleaned_table = [["" if cell is None else str(cell) for cell in row] for row in table]
                    formatted_table = "\n".join(["\t".join(row) for row in cleaned_table if any(row)])
                    table_content.append(formatted_table)

    combined_text = "\n\n".join(text_content)
    combined_tables = "\n\n".join(table_content)

    return combined_text, combined_tables

# Function to split documents
def split_docs(docs, chunk_size=1000, chunk_overlap=20):
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunks = splitter.split_documents(documents=docs)
    return chunks

# Embedding model loader with CUDA support
def load_embedding_model(model_path, normalize_embedding=True):
    return HuggingFaceEmbeddings(
        model_name=model_path,
        model_kwargs={'device': device},  # Use the selected device
        encode_kwargs={'normalize_embeddings': normalize_embedding}
    )

# Create embeddings
def create_embeddings(chunks, embedding_model, storing_path='vectorstore'):
    vectorstore = FAISS.from_documents(chunks, embedding_model)
    vectorstore.save_local(storing_path)
    return vectorstore

# Updated prompt template
template = '''
    You are a technical assistant specialized in handling welding equipment, specifically the Warrior 500i CC/CV.
    You provide precise and accurate responses based on the manual, focusing on technical data.
    When responding to user queries, follow these instructions:

    - If the user asks about **safety**, make sure to include any relevant standards like ANSI/ASC Standard Z49.1.
    - If the user asks for a **full section or Table of Contents**, provide the complete content without omissions.
    - For longer content, make sure the response is split clearly into sub-sections for readability.
    - If the user asks for a **list of types, examples, or uses**, provide them in a bullet-point format.
    - If the user asks for **specific details or explanations**, provide them in a structured paragraph and some bullet points format.
    - If the user requests a **step-by-step guide**, break down the information into numbered steps.
    - If the user asks for **troubleshooting or errors**, list the errors and their corresponding solutions clearly, making sure not to skip any points.
    - When providing **key points** for a query response provide as bullet points.
    - For **comparisons**, ensure to highlight the differences and similarities clearly.

    Always ensure that the response is clear, concise, and contextual. Use the previous chat responses to maintain relevance but avoid unnecessary repetition.

    "{context}"

    "### User:"
    "{question}"

    ### Response:
'''

# Function to format responses
def format_response(response_text, user_query):
    # Keywords for different formats
    list_keywords = ['list', 'types', 'examples', 'uses', 'features', 'characteristics', 'mention', 'provide']
    step_keywords = ['steps', 'how to', 'process', 'procedure', 'guide']
    error_keywords = ['error', 'troubleshooting', 'fault']

    # Check if the question indicates a list or step-by-step format
    if any(keyword in user_query.lower() for keyword in list_keywords):
        response_lines = response_text.split('\n')
        formatted_response = "\n\n".join([f"- {line.strip()}" for line in response_lines if line.strip()])

    elif any(keyword in user_query.lower() for keyword in step_keywords):
        # Format response as numbered steps
        response_lines = response_text.split('\n')
        formatted_response = "\n\n".join([f"{idx + 1}. {line.strip().lstrip('1234567890. ')}" for idx, line in enumerate(response_lines) if line.strip()])

    elif any(keyword in user_query.lower() for keyword in error_keywords):
        # Specific format for errors in a troubleshooting context
        response_lines = response_text.split('\n')
        formatted_response = "\n\n".join([f"**{line.strip()}**" if "Error" in line else f"  - {line.strip()}" for line in response_lines if line.strip()])

    else:
        # Default to paragraph format for detailed explanations
        formatted_response = textwrap.fill(response_text, width=6000)

    return formatted_response

# QA Chain Loader with CUDA support
def load_qa_chain(retriever, llm, prompt):
    return RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,
        chain_type='stuff',
        return_source_documents=True,
        chain_type_kwargs={'prompt': prompt}
    )

# Load embedding model
embed = load_embedding_model(model_path="sentence-transformers/all-MiniLM-L6-v2")

# Load PDF and extract data
file_path = "/content/Warrior 500i.pdf"
text_data, table_data = extract_text_and_tables_from_pdf(file_path)

# Combine content into Document structure
combined_content = [Document(page_content=text_data, metadata={'source': 'Text Data'}),
                    Document(page_content=table_data, metadata={'source': 'Table Data'})]

# Split documents into chunks
documents = split_docs(combined_content)

# Create embeddings and vector store
vectorstore = create_embeddings(documents, embed)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# Create prompt
prompt = PromptTemplate.from_template(template)
chain = load_qa_chain(retriever, llm, prompt)

# Function to add chat bubbles with custom styles
def add_chat_bubble(role, text):
    if role == 'user':
        # User's message style
        st.markdown(
            f"""
            <div style='background-color: #f0f0f0; border-radius: 15px; padding: 10px; margin-bottom: 10px; width: fit-content; max-width: 70%; margin-left: auto;'>
                <strong style='color: #e74c3c;'>🧑</strong>
                <span style='margin-left: 10px;'>{text}</span>
            </div>
            """,
            unsafe_allow_html=True
        )
    elif role == 'bot':
        # Bot's message style
        st.markdown(
            f"""
            <div style='background-color: #e8f8e8; border-radius: 15px; padding: 10px; margin-bottom: 10px; width: fit-content; max-width: 70%;'>
                <strong style='color: #3498db;'>🤖</strong>
                <span style='margin-left: 10px;'>{text}</span>
            </div>
            """,
            unsafe_allow_html=True
        )

# Streamlit UI setup
st.title("Warrior 500i Chatbot")

user_input = st.text_input("Ask a question about Warrior 500i equipment")

greetings = ['hi', 'hello', 'hey', 'hola', 'howdy', 'greetings']

if user_input:
    try:
        if any(greet in user_input.lower() for greet in greetings) and len(user_input.split()) <= 3 and not st.session_state['greeting_given']:
            bot_response = "Hello! How may I assist you today?"
            st.session_state['greeting_given'] = True
        else:
            # Build up chat history to pass as context
            chat_history = "\n\n".join([f"User: {item['user']}\nBot: {item['bot']}" for item in st.session_state['chat_history']])
            context = chat_history + f"\n\n### New Query:\nUser: {user_input}\nBot:"

            # Get the response from the LLM with context
            response = chain({'query': user_input, 'context': context})
            formatted_response = format_response(response['result'], user_input)
            bot_response = formatted_response

        # Update the chat history with the current interaction
        st.session_state['chat_history'].append({'user': user_input, 'bot': bot_response})

    except Exception as e:
        st.error(f"An error occurred: {e}")

# Display chat history with chat bubbles
if st.session_state['chat_history']:
    for chat in st.session_state['chat_history']:
        add_chat_bubble('user', chat['user'])
        add_chat_bubble('bot', chat['bot'])


Overwriting app.py


In [8]:
from pyngrok import ngrok

!ngrok config add-authtoken 2jVXhdtDG2EtZtChj2aqD90R0rU_5WMs9eDcGsVuP4jiSi5kV

# Start Streamlit in the background
!streamlit run app.py &

# Create a public URL using ngrok
public_url = ngrok.connect(port='8501')
print(f"Public URL for Streamlit App: {public_url}")


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.204.39.218:8501

  Stopping...


PyngrokNgrokHTTPError: ngrok client exception, API returned 400: {"error_code":102,"status_code":400,"msg":"invalid tunnel configuration","details":{"err":"yaml: unmarshal errors:\n  line 1: field port not found in type config.HTTPv2Tunnel"}}


In [ ]:
# import os
# import streamlit as st
# from langchain_community.embeddings import HuggingFaceEmbeddings
# from langchain_community.document_loaders import PyMuPDFLoader
# from langchain_community.vectorstores import FAISS
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.chains import RetrievalQA
# from langchain_community.llms import Ollama
# from langchain_core.prompts import PromptTemplate
# from langchain.schema import Document
# import pdfplumber
# import textwrap
# import warnings

# warnings.filterwarnings("ignore")

# llm = Ollama(model='llama3.1', temperature=0.05)

# # Initialize chat history in session state
# if 'chat_history' not in st.session_state:
#     st.session_state['chat_history'] = []

# if 'greeting_given' not in st.session_state:
#     st.session_state['greeting_given'] = False

# # List of common greetings to handle initial conversations
# greetings = ['hi', 'hello', 'hey', 'hola', 'howdy', 'greetings']

# # Improved function to extract text and tables from PDF
# def extract_text_and_tables_from_pdf(file_path):
#     text_content = []
#     table_content = []

#     with pdfplumber.open(file_path) as pdf:
#         for page in pdf.pages:
#             # Extract page text
#             text = page.extract_text()
#             if text:
#                 text_content.append(text)

#             # Extract tables
#             tables = page.extract_tables()
#             for table in tables:
#                 if table:
#                     # Clean and verify table data
#                     cleaned_table = [["" if cell is None else str(cell) for cell in row] for row in table]
#                     formatted_table = "\n".join(["\t".join(row) for row in cleaned_table if any(row)])
#                     table_content.append(formatted_table)

#     combined_text = "\n\n".join(text_content)
#     combined_tables = "\n\n".join(table_content)

#     return combined_text, combined_tables

# # Function to split documents
# def split_docs(docs, chunk_size=1000, chunk_overlap=20):
#     splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
#     chunks = splitter.split_documents(documents=docs)
#     return chunks

# # Embedding model loader
# def load_embedding_model(model_path, normalize_embedding=True):
#     return HuggingFaceEmbeddings(
#         model_name=model_path,
#         model_kwargs={'device': 'cpu'},
#         encode_kwargs={'normalize_embeddings': normalize_embedding}
#     )

# # Create embeddings
# def create_embeddings(chunks, embedding_model, storing_path='vectorstore'):
#     vectorstore = FAISS.from_documents(chunks, embedding_model)
#     vectorstore.save_local(storing_path)
#     return vectorstore

# # Your Prompt Template
# template = '''
#     You are a technical assistant specialized in handling welding equipment, specifically the Warrior 500i CC/CV.
#     You provide precise and accurate responses based on the manual, focusing on technical data.
#     When responding to user queries, follow these instructions:

#     - If the user asks about **safety**, make sure to include any relevant standards like ANSI/ASC Standard Z49.1.
#     - If the user asks for a **full section or Table of Contents**, provide the complete content without omissions.
#     - For longer content, ensure the response is split clearly into sub-sections for readability, each sub-section with a clear heading.
#     - If the user asks for a **list of types, examples, or uses**, provide them as bullet points, with each item on a new line.
#     - If the user asks for **specific details or explanations**, write them in concise paragraphs for general descriptions. For technical specifications, use bullet points or numbered lists, each point on a new line.
#     - If the user requests a **step-by-step guide**, break down the information into numbered steps, each step on a new line.
#     - If the user asks for **troubleshooting or errors**, list errors and corresponding solutions separately, each on its own line.
#     - When providing **key points**, ensure each point is formatted on a new line, using bullet points.
#     - For **comparisons**, highlight differences and similarities clearly, using bullet points if necessary, with each comparison on a new line.
#     - **Numbered or Bullet-Point Information**: Use bullet points for any list or technical details. Each bullet point should be on a new line. Avoid grouping bullet points into paragraphs.
#     - **Paragraphs**: Use only for continuous descriptions without any bullet points or technical lists.

#     Ensure all responses are structured for clarity, using proper line breaks and avoiding clumped information. Use the previous chat responses to maintain context but avoid unnecessary repetition.

#     "{context}"

#     "### User:"
#     "{question}"

#     ### Response:
# '''

# # Function to format responses
# def format_response(response_text, user_query):
#     # Keywords for different formats
#     list_keywords = ['list', 'types', 'examples', 'uses', 'features', 'characteristics', 'mention', 'provide']
#     step_keywords = ['steps', 'how to', 'process', 'procedure', 'guide']
#     error_keywords = ['error', 'troubleshooting', 'fault']

#     # Split the response text by lines
#     response_lines = response_text.split('\n')

#     # Determine formatting based on the user query
#     if any(keyword in user_query.lower() for keyword in list_keywords):
#         # Format response as bullet points
#         formatted_response = "\n\n".join([f"• {line.strip()}" for line in response_lines if line.strip()])

#     elif any(keyword in user_query.lower() for keyword in step_keywords):
#         # Format response as numbered steps
#         formatted_response = "\n\n".join([f"{idx + 1}. {line.strip().lstrip('1234567890. ')}" for idx, line in enumerate(response_lines) if line.strip()])

#     elif any(keyword in user_query.lower() for keyword in error_keywords):
#         # Specific format for errors in a troubleshooting context
#         formatted_response = "\n\n".join([f"**{line.strip()}**" if "Error" in line else f"  - {line.strip()}" for line in response_lines if line.strip()])

#     else:
#         # Default to handling each bullet or numbered line separately
#         formatted_lines = []
#         for line in response_lines:
#             stripped_line = line.strip()
#             if stripped_line and (stripped_line.startswith(('*', '-', '•')) or
#                                   (len(stripped_line) > 1 and stripped_line[0].isdigit() and stripped_line[1] in ['.', ')'])):
#                 # Bullet or numbered point, keep on a new line with a bullet
#                 formatted_lines.append(f"• {stripped_line.lstrip('*-.• ')}")
#             else:
#                 # Treat as regular paragraph if not a list item
#                 if stripped_line:  # Ensure it's not empty
#                     # Wrap text within a width of 80 characters
#                     wrapped_text = textwrap.fill(stripped_line, width=4000)
#                     formatted_lines.append(wrapped_text)

#         # Join lines with proper line breaks
#         formatted_response = "\n\n".join(formatted_lines)

#     return formatted_response

# # QA Chain Loader
# def load_qa_chain(retriever, llm, prompt):
#     return RetrievalQA.from_chain_type(
#         llm=llm,
#         retriever=retriever,
#         chain_type='stuff',
#         return_source_documents=True,
#         chain_type_kwargs={'prompt': prompt}
#     )

# # Load embedding model
# embed = load_embedding_model(model_path="sentence-transformers/all-MiniLM-L6-v2")

# # Load PDF and extract data
# file_path = "Warrior 500i.pdf"
# text_data, table_data = extract_text_and_tables_from_pdf(file_path)

# # Combine content into Document structure
# combined_content = [Document(page_content=text_data, metadata={'source': 'Text Data'}), Document(page_content=table_data, metadata={'source': 'Table Data'})]

# # Split documents into chunks
# documents = split_docs(combined_content)

# # Create embeddings and vector store
# vectorstore = create_embeddings(documents, embed)
# retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# # Create prompt
# prompt = PromptTemplate.from_template(template)
# chain = load_qa_chain(retriever, llm, prompt)

# # Chat history management
# def get_chat_history():
#     chat_history_str = ""
#     for chat in st.session_state['chat_history']:
#         chat_history_str += f"User: {chat['user']}\nBot: {chat['bot']}\n"
#     return chat_history_str.strip()

# # Function to add chat bubbles with custom styles
# def add_chat_bubble(role, text):
#     if role == 'user':
#         # User's message style
#         st.markdown(
#             f"""
#             <div style='background-color: #f0f0f0; border-radius: 15px; padding: 10px; margin-bottom: 10px; width: fit-content; max-width: 70%; margin-left: auto;'>
#                 <strong style='color: #e74c3c;'>🧑</strong>
#                 <span style='margin-left: 10px;'>{text}</span>
#             </div>
#             """,
#             unsafe_allow_html=True
#         )
#     elif role == 'bot':
#         # Bot's message style
#         st.markdown(
#             f"""
#             <div style='background-color: #e8f8e8; border-radius: 15px; padding: 10px; margin-bottom: 10px; width: fit-content; max-width: 70%;'>
#                 <strong style='color: #3498db;'>🤖</strong>
#                 <span style='margin-left: 10px;'>{text}</span>
#             </div>
#             """,
#             unsafe_allow_html=True
#         )

# # Streamlit UI setup
# st.title("Warrior 500i Chatbot")

# # Display chat history initially
# if st.session_state['chat_history']:
#     for chat in st.session_state['chat_history']:
#         add_chat_bubble('user', chat['user'])
#         add_chat_bubble('bot', chat['bot'])

# # Input box for user question
# user_input = st.text_input("Ask a question about Warrior 500i equipment", key="user_input")

# # Buttons for 'Send' and 'Clear Chat' under the input field
# col1, col2, col3 = st.columns([1, 5, 1])

# with col1:  # Clear Chat button on the left
#     st.markdown(
#         """
#         <style>
#         .stButton > button {
#             width: 120px;
#             height: 40px;
#             padding: 10px;
#             margin: 5px 0;
#             text-align: center;
#             font-size: 14px;
#         }
#         </style>
#         """,
#         unsafe_allow_html=True
#     )
#     clear_button = st.button("Clear Chat")

# # Send button remains as it is on the right
# with col3:
#     send_button = st.button("Send")

# # Clear chat functionality
# if clear_button:
#     st.session_state['chat_history'] = []
#     st.session_state['greeting_given'] = False
#     st.experimental_rerun()  # Refresh the app to reset the UI

# # Print user input for debugging purposes
# print(f'User Input: {user_input}')

# # Handling the Send button click
# if send_button and user_input:
#     try:
#         if any(greet in user_input.lower() for greet in greetings) and len(user_input.split()) <= 3 and not st.session_state['greeting_given']:
#             bot_response = "Hello! How may I assist you today?"
#             st.session_state['greeting_given'] = True
#         else:
#             # Build up chat history to pass as context
#             chat_history = get_chat_history()
#             context = chat_history + f"\n\n### New Query:\nUser: {user_input}\nBot:"

#             # Get the response from the LLM with context
#             response = chain({'query': user_input, 'context': context})
#             formatted_response = format_response(response['result'], user_input)
#             bot_response = formatted_response
#             print(f"Bot Response: {bot_response}")

#         # Update the chat history with the current interaction
#         st.session_state['chat_history'].append({'user': user_input, 'bot': bot_response})

#         # Refresh chat display after appending new interaction
#         add_chat_bubble('user', user_input)
#         add_chat_bubble('bot', bot_response)

#     except Exception as e:
#         st.error(f"An error occurred: {e}")
